In [1]:
import pandas as pd
import numpy as np
import random
import os
import gc
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv('C:/Users/jamos/OneDrive - UOU/2024-winter_semester/Dacon/Seoul_degree_preview/open/train.csv')
sub = pd.read_csv('C:/Users/jamos/OneDrive - UOU/2024-winter_semester/Dacon/Seoul_degree_preview/open/sample_submission.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23011 entries, 0 to 23010
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일시      23011 non-null  object 
 1   최고기온    23008 non-null  float64
 2   최저기온    23008 non-null  float64
 3   일교차     23007 non-null  float64
 4   강수량     9150 non-null   float64
 5   평균습도    23011 non-null  float64
 6   평균풍속    23007 non-null  float64
 7   일조합     22893 non-null  float64
 8   일사합     18149 non-null  float64
 9   일조율     22645 non-null  float64
 10  평균기온    23011 non-null  float64
dtypes: float64(10), object(1)
memory usage: 1.9+ MB


강수량, 일사합 에서 특징적으로 Null 값 확인

# 최고 기온 SELF

In [4]:
from prophet import Prophet

# 날짜 데이터 변환
train['일시'] = pd.to_datetime(train['일시'])
train_df = train.set_index('일시')

# 데이터의 시간 간격 지정
train_df.index.freq = 'D'   

# 일시 컬럼이 인덱스로 할당됩니다.
train_df.head()


# 각 칼럼의 평균값 계산
mean_values = train_df.mean()

# NaN 값을 각 컬럼의 평균값으로 대체
train_df.fillna(mean_values, inplace = True)    
train_df = train_df.reset_index()

평균기온 데이터 들의 값 더하거나 뻄

In [7]:
# '평균기온'이 1월의 평균 기온보다 높거나 같은 모든 행을 찾습니다. 그런 다음 해당 행들의 '평균기온' 값에 1.5를 더합니다.
df_january = train_df[train_df['일시'].dt.month ==1]
january_avg_temp = df_january['평균기온'].mean()

## 평균 기온 이하인 날짜에 대해 평균기온에 +1을 추가
train_df.loc[(train_df['일시'].dt.month == 1) & (train_df['평균기온'] >= january_avg_temp), '평균기온'] += 2.1

# 2월의 평균 기온 계싼
df_febuary = train_df[train_df['일시'].dt.month == 2]
febuary_avg_temp = df_febuary['평균기온'].mean()

## 평균 기온이 2월의 평균 기온 보다 높거나 같은 경우 평균 기온에 2.2 추가
train_df.loc[(train_df['일시'].dt.month ==2) & (train_df['평균기온'] >=febuary_avg_temp), '평균기온'] += 2.7

# 3월의 평균 기온 계산
df_march = train_df[train_df['일시'].dt.month ==3]
march_avg_temp = df_march['평균기온'].mean()

## 평균 기온이 3월의 평균 기온보다 높거나 같은 걍우 평균기온에 2.1 추가
train_df.loc[(train_df['일시'].dt.month == 3) & (train_df['평균기온'] >= march_avg_temp), '평균기온'] += 5.6

# 8월의 평균 기온 계산
df_august = train_df[train_df['일시'].dt.month == 8]    
august_avg_temp = df_august['평균기온'].mean()

## 평균 기온이 8월의 평균 기온보다 높거나 같은 경우 평균기온에 1 추가
train_df.loc[(train_df['일시'].dt.month == 8) &(train_df['평균기온'] >= august_avg_temp), '평균기온'] +=2

# 9월의 평균 기온 계산
df_september = train_df[train_df['일시'].dt.month == 9] 
september_avg_temp = df_september['평균기온'].mean()

# 평균 기온이 9월의 평균 기온보다 높거나 같은 경우 평균기온에 1 추가
train_df.loc[(train_df['일시'].dt.month == 9) & (train_df['평균기온'] >= september_avg_temp), '평균기온'] += 2

# 10월의  평균 기온 계산
df_october = train_df[train_df['일시'].dt.month == 10]
october_avg_temp = df_october['평균기온'].mean()

# 평균 기온이 10월의 평균 기온보다 높거나 같은 경우 평균기온에 1 추가
train_df.loc[(train_df['일시'].dt.month == 10) & (train_df['평균기온'] >= october_avg_temp), '평균기온'] -= 1.5

# 11월의 평균 기오 ㄴ계산
df_november = train_df[train_df['일시'].dt.month == 11]
november_avg_temp = df_november['평균기온'].mean()

# 평균 기온이 11월의 평균 기온 바다 높거나 갗은 경우 평균 기온ㅇ에 1 추가
train_df.loc[(train_df['일시'].dt.month == 11) & (train_df['평균기온'] >= november_avg_temp), '평균기온'] -=3.1

# 12월의 평균 기온 계산
df_december = train_df[train_df['일시'].dt.month == 12]
december_avg_temp = df_december['평균기온'].mean()

# 평균 기온이 12월의 평균 기온보다 높거나 같은 경우 평균 기온에 1 추가
train_df.loc[(train_df['일시'].dt.month == 12) & (train_df['평균기온'] >= december_avg_temp), '평균기온'] -= 4.5

########################################333

# 5월의 평균 기온 계산
df_may = train_df[train_df['일시'].dt.month == 5]
may_avg_temp = df_may['평균기온'].mean()

# 평균 기온이 5월의 평균 기온보다 높거나 같은 경우 평균 기온에 1 추가
train_df.loc[(train_df['일시'].dt.month == 5) & (train_df['평균기온'] >= may_avg_temp), '평균기온'] += 1


In [8]:
# 날짜 데이터 변환
sub['일시'] = pd.to_datetime(sub['일시'])
sub = sub.set_index('일시')

# 데이터의 시간 간격 지정
sub.index.freq = 'D'

# 일시 컬럼이 인덱스로 할당됩니다.
sub.head()

sub = sub.reset_index()


In [9]:
# prophet 에서 데이터를 인식하도록 일시는 ds 로 , target값인 평균기온은 y 로 지정해줍니다.
train_df = train_df.rename(columns = {'일시' : 'ds', '최고기온': 'y'})

In [10]:
train_df['일'] = train_df['ds'].dt.day
sub['일'] = sub['일시'].dt.day

In [11]:
train_df['달'] = train_df['ds'].dt.month
sub['달'] = sub['일시'].dt.month

In [12]:
# initialize the Prophet model with logistic growth
m = Prophet(seasonality_prior_scale = 0.1, n_changepoints = 40)
m.add_seasonality(name = 'semiannual', period = 180, fourier_order = 10)
m.add_regressor('일')
m.add_regressor('달')
m.fit(train_df)

19:59:42 - cmdstanpy - INFO - Chain [1] start processing
19:59:54 - cmdstanpy - INFO - Chain [1] done processing


In [13]:
future_data = m.make_future_dataframe(periods = 358, freq = 'd')

In [14]:
import pandas as pd

# '최저기온' 데이터를 train_df 와 sub 에서 추출
train_temps = train_df['일']
sub_temps = sub['일']

# 데이터 결합
combined_temps = pd.concat([train_temps, sub_temps])

# future_data 에 '최저 기온' 추가
future_data['일']    = combined_temps.values

###########################################33
import pandas as pd

# '최저 기온 ' 데이터를 train_df 와 sub 에서 추출
train_temps = train_df['달']
sub_temps = sub['달']

# 데이터 결합
combined_temps = pd.concat([train_temps, sub_temps])

# future_data 에 '최저기온' 추가
future_data['달'] = combined_temps.values
###################################################33

In [15]:
forecast_data = m.predict(future_data)  
forecast_data[['ds', 'yhat']].tail(5)

,ds,yhat
23364,2023-12-20,4.710338
23365,2023-12-21,4.644573
23366,2023-12-22,4.534165
23367,2023-12-23,4.508127
23368,2023-12-24,4.388631


In [16]:
sub['최고기온'] = forecast_data.yhat[-358:].values

In [17]:
sub

,일시,평균기온,일,달,최고기온
0,2023-01-01,0,1,1,3.903483
1,2023-01-02,0,2,1,3.772475
2,2023-01-03,0,3,1,3.685933
3,2023-01-04,0,4,1,3.575172
4,2023-01-05,0,5,1,3.524494
...,...,...,...,...,...
353,2023-12-20,0,20,12,4.710338
354,2023-12-21,0,21,12,4.644573
355,2023-12-22,0,22,12,4.534165
356,2023-12-23,0,23,12,4.508127


In [18]:
sub = sub.drop(columns = ['일'])

In [19]:
sub = sub.drop(columns = ['달'])

In [20]:
sub

,일시,평균기온,최고기온
0,2023-01-01,0,3.903483
1,2023-01-02,0,3.772475
2,2023-01-03,0,3.685933
3,2023-01-04,0,3.575172
4,2023-01-05,0,3.524494
...,...,...,...
353,2023-12-20,0,4.710338
354,2023-12-21,0,4.644573
355,2023-12-22,0,4.534165
356,2023-12-23,0,4.508127
